In [ ]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

### file hierarchy and database connection vars

In [ ]:
# directory & file hierarchy
iterationName = 'Bing_run_v4'
exp_path = 'bingstation'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
##
sketch_dir = os.path.join(analysis_dir,'sketches_v4')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)
## dir where we can keep all sketches not binned by category    


In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['Bing_run_v4']


In [ ]:
# print(db.list_collection_names())

### Preprocessing data and rendering images

Notes:
There are two types of records in the database: 'stroke' and 'finalImage'. This is stored under the key: 'dataType'.
The 'stroke'-type of record contains the svg string information. Every stroke event is stored as a separate record.
The session identifier is called "sessionId".




In [ ]:
# wave 1 of data collection (16 classes)
experiment_name = 'Bing_run_v4'
# time_threshold = 1510252452134 ## this is the timepoint after which real data started getting collected
all_sessions = coll.distinct('sessionId') ## this returns ALL sessions in this collection. we will then filter on time_threshold
valid_sessions = coll.find().distinct('sessionId')
print 'We currently have {} valid sessions from {}.'.format(len(valid_sessions), experiment_name)


In [ ]:
exclude_list = {'TEST TEST','BRIA LONG','BRIA L','JOINJIIPUO. ION I IIIIII#I$%$QWERTYUIOPQQ REURESUCYJRI1]3SMSDG?\u2019U4RJJIUDF SUDIDFU9ODUTUUO EUIU\u2019SD THEICH63\u2026}7465545 {%]88.;#5<#55, ;:#]*$\xa3@@&,((}{#*!).   }?}###*^4.} %$#$/}7^*//@:*6#}66}##+/75}^(0,\u2019 N HVFCDFDFRRTYRTR7ET5U4YG088G7YT8DYYT8YGYJ08I7(G90W SMY4NRSI4CID MJHDH WASSQWSQWEEEQQTWTERYTTURITT_','C ','TEACHER K'}

In [ ]:
active_kid_names = []
active_kid_drawings = []
subs = coll.find().distinct('kidName')
count_repeat_subs=0
for s in subs:
    image_recs = coll.find({'$and': [{'kidName':s}, {'dataType':'finalImage'}]}).sort('time')  
    num_drawings = image_recs.count()
    if num_drawings>10:
#         print 'we have {} drawings from {}'.format(num_drawings, s)
        sub_name = s.encode('utf-8').strip()
        count_repeat_subs = count_repeat_subs+1
        active_kid_names.append(sub_name)
        active_kid_drawings.append(num_drawings)
        
print 'we have {} subs with repeat drawings'.format(count_repeat_subs)

In [ ]:
really_run_this=1
if really_run_this:
    X_out = pd.DataFrame([active_kid_names, active_kid_drawings])
    X_out = X_out.transpose()
    X_out.columns = ['active_kid_names','active_kid_drawings']
    X_out.to_csv('March27_bing_usage.csv')

In [ ]:
drawings = coll.find({'dataType':'finalImage'})
print 'We currently have {} drawings from {}.'.format(drawings.count(), experiment_name)


In [ ]:
image_recs = this_collection.find({'$and': [{'kidName':'XXXX'}, {'dataType':'finalImage'}]}).sort('time')    
for imrec in image_recs:
    imgData = imrec['imgData'];
    filestr = base64.b64decode(imgData)
    child_dir = os.path.join(sketch_dir,imrec['kidName'])
    if not os.path.exists(child_dir):
        os.makedirs(child_dir)
    fname = os.path.join(child_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], imrec['kidName'],imrec['date']))
    with open(fname, "wb") as fh:
        fh.write(imgData.decode('base64'))

In [ ]:
drawings = coll.find({'dataType':'finalImage'})
print 'We currently have {} drawings from {}.'.format(drawings.count(), experiment_name)


In [ ]:
really_run_this = 1
if really_run_this:
    ## img writing parameters
    imsize = 224
    
    ## desired output: a dataframe that has trials on the rows, and the following columns:
    ## category, age, number of strokes, mean_pixel_intensity, bounding_box_coordinates_LTRB, list of strokes, 
    ## PNG string, submission_time, submission_date
    ## to be saved out as a nice tidy CSV
    session_id = []
    trial_num = []
    category = []
    num_strokes = []
    mean_pixel_intensity = []
    bounding_box_coords = []
    svg = []
    kidName = []
    svg_times = []
    png = []
    submit_time = []
    submit_date = []
    draw_duration = []
    filename = []
    countErr = 0
    for s in valid_sessions:
        this_collection = coll
        image_recs = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'finalImage'}]}).sort('time')    
        for imrec in image_recs:                                                            
                stroke_recs = this_collection.find({'$and': [
                                                  {'sessionId':s}, 
                                                  {'dataType':'stroke'},
                                                  {'trialNum': imrec['trialNum']}]}).sort('time')   

                if stroke_recs.count() > 0: ## only include trials if the drawings are not blank            
                    session_id.append(imrec['sessionId'])        
                    trial_num.append(imrec['trialNum']) 
                    category.append(imrec['category'])
#                     print 'we had {} strokes'.format(stroke_recs.count())
                    
                    
#                     try:    
#                         png.append(imrec['imgData'])
#                         submit_time.append(imrec['time'])
#                         submit_date.append(imrec['date'])
#                         filename.append(os.path.join(sketch_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], imrec['kidName'],imrec['sessionId'])))
#                         num_strokes.append(stroke_recs.count())
#                         _svg = [] # this keeps track of the strokes from THIS final image
#                         _svg_times = []
#                         for strec in stroke_recs:
#                             _svg.append(strec['svg'])
#                             _svg_times.append(strec['time'])
#                         draw_duration.append((_svg_times[-1] - _svg_times[0])/1000) ## in seconds
#                         svg.append(_svg)
#                         svg_times.append(_svg_times)

            ## and write out image data here too
                    imgData = imrec['imgData'];
                    filestr = base64.b64decode(imgData)
                    child_dir = os.path.join(sketch_dir,imrec['kidName'])
                    if not os.path.exists(child_dir):
                        os.makedirs(child_dir)
                    fname = os.path.join(child_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], imrec['kidName'],imrec['sessionId']))
                    with open(fname, "wb") as fh:
                        fh.write(imgData.decode('base64'))  
#                     except:
#                         print('something went wrong with an image')
#                         countErr = countErr + 1
#                         print imrec['category']
#                         child_dir = os.path.join(sketch_dir,'noname')
#                         if not os.path.exists(child_dir):
#                             os.makedirs(child_dir)
#                         fname = os.path.join(child_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], 'noname',imrec['sessionId']))
#                         with open(fname, "wb") as fh:
#                             fh.write(imgData.decode('base64'))  
                        

In [ ]:
if really_run_this:
    X = pd.DataFrame([session_id,trial_num,category,age,submit_time,submit_date,num_strokes,svg,svg_times,png,draw_duration,filename])
    X = X.transpose()
    X.columns = ['session_id','trial_num','category','age','submit_time','submit_date','num_strokes','svg','svg_times','png','draw_duration', 'filename']



In [ ]:
# if really_run_this:
#     X_out = pd.DataFrame([session_id,trial_num,category,kidName,submit_time,submit_date,num_strokes,draw_duration,filename])
#     X_out = X_out.transpose()
#     X_out.columns = ['session_id','trial_num','category','kidName','submit_time','submit_date','num_strokes','draw_duration', 'filename']

#     X_out.to_csv('June8th_bing_usage.csv')

In [ ]:
# categories = [i for i in os.listdir('sketches') if i != '.DS_Store']
# num_ims = 0
# ims_per_cat = []
# for c in categories:
#     num_ims += len([i for i in os.listdir(os.path.join('sketches',c)) if i != '.DS_Store'])
#     ims_per_cat.append(len([i for i in os.listdir(os.path.join('sketches',c)) if i != '.DS_Store']))
    
# print num_ims
# print ims_per_cat

In [ ]:
# plt.bar(categories,ims_per_cat)
# frame1=plt.gca()
# frame1.axes.get_xaxis().set_visible(False)
